In [ ]:
import sys
sys.path.append("..")
sys.path.append("my_library")
from my_library.config import *
from my_library.metrics.cmp_metrics import *
sys.path.append("..")
import os

n_classes = 2

import os

#%run -i visualization/compute_metrics_single_run.ipynb

In [ ]:
import pandas as pd
csv_df = pd.read_csv(working_directory+"/projects/I3lung-sqadqc-project/annotations.csv")


In [ ]:
import pandas as pd

# Install requirements if needed:
# pip install pyarrow

# Read the Parquet file
path = working_directory+"/eval/train/pdl1-3/inner_iteration_1/00000-attention_mil"
df_predictions = pd.read_parquet(path + "/predictions.parquet")

# Explore the data
print(df_predictions.head())
print(df_predictions.info())

df_predictions

In [ ]:
csv_df

In [ ]:
df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')

In [ ]:
df_joined

In [ ]:


# Distribution of values in 'category'
distribution = df_joined["PDL1_CATHEGORY"].value_counts()
print(distribution)


In [ ]:
import os

root_dir = "../eval/train/pdl1-3"

train_parquet_paths = []
val_parquet_paths = []
test_parquet_paths = []

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                train_parquet_paths.append(file_path)
train_parquet_paths.sort()

#print("Sorted Parquet paths:")
#for path in train_parquet_paths:
#    print(path)

root_dir = "../eval/val/pdl1-3"

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                val_parquet_paths.append(file_path)
val_parquet_paths.sort()


root_dir = "../eval/test-int/pdl1-3"

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if the folder name contains "attention"
    if "attention" in os.path.basename(dirpath).lower():

        # If you want to do something with each file inside those folders
        for f in filenames:
            file_path = os.path.join(dirpath, f)
            if f.lower().endswith(".parquet"):
                #print(f"   This is a Parquet file: {file_path}")
                test_parquet_paths.append(file_path)
test_parquet_paths.sort()


In [ ]:
val_parquet_paths

In [ ]:
test_parquet_paths

# Extracting the names of the files

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.metrics import classification_report, accuracy_score

In [ ]:
metrics_list = []
numbers = []
desired_order = ["< 1 %", "1-49 %", ">=50 %"]

train_models = []
train_classification_reports = []
for i in train_parquet_paths:
    df_predictions = pd.read_parquet(i)
    df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
    #print(df_joined)
    distribution = df_joined[["slide", "PDL1_CATHEGORY"]]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    x_list = np.ones((distribution.shape[0], 1))
    y_list = df_joined["PDL1_CATHEGORY"].to_numpy()

    dummy_clf = DummyClassifier(strategy="most_frequent")
    dummy_clf.fit(x_list, y_list)

    y_pred = dummy_clf.predict(x_list)

    train_models.append(dummy_clf)

    # Accuratezza generale
    acc = accuracy_score(y_list, y_pred)
    #print(acc)

    # Report dettagliato per classe
    report = classification_report(y_list, y_pred, digits=2, output_dict=True, zero_division=0)

    #print(report)

    train_classification_reports.append(report)

#df_train = pd.DataFrame(metrics_list, numbers)
#display(distribution)


In [ ]:
train_models

In [ ]:
train_classification_reports

In [ ]:
import numpy as np

# Assuming your list of dictionaries is stored in the variable `reports`
reports = train_classification_reports  # your list of dictionaries

def average_report_group(group):
    """Average a group of report dictionaries."""
    avg_report = {}
    # Loop over each key in the first dictionary of the group.
    for key in group[0]:
        if key == 'accuracy':
            # Average the accuracy values directly.
            avg_report[key] = np.mean([r[key] for r in group])
        else:
            # For keys that are nested dictionaries, average each inner metric.
            inner_keys = group[0][key].keys()
            avg_inner = {}
            for metric in inner_keys:
                avg_inner[metric] = np.mean([r[key][metric] for r in group])
            avg_report[key] = avg_inner
    return avg_report

group_size = 5
averaged_reports = []

# Process the reports list in chunks of 5.
for i in range(0, len(reports), group_size):
    group = reports[i:i + group_size]
    avg = average_report_group(group)
    averaged_reports.append(avg)

# Now `averaged_reports` contains the averaged dictionary for each group of 5 items.
print(averaged_reports)

df_averaged_pred = pd.DataFrame(averaged_reports)


In [ ]:
display(df)

In [ ]:
metrics_list = []
numbers = []
desired_order = ["< 1 %", "1-49 %", ">=50 %"]

val_models = []
val_classification_reports = []
for i in val_parquet_paths:
    df_predictions = pd.read_parquet(i)
    df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
    #print(df_joined)
    distribution = df_joined[["slide", "PDL1_CATHEGORY"]]

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    x_list = np.ones((distribution.shape[0], 1))
    y_list = df_joined["PDL1_CATHEGORY"].to_numpy()

    dummy_clf = DummyClassifier(strategy="most_frequent")
    dummy_clf.fit(x_list, y_list)

    y_pred = dummy_clf.predict(x_list)

    val_models.append(dummy_clf)

    # Accuratezza generale
    acc = accuracy_score(y_list, y_pred)
    print(acc)

    # Report dettagliato per classe
    report = classification_report(y_list, y_pred, digits=2, output_dict=True, zero_division=0)

    print(report)

    val_classification_reports.append(report)

#df_train = pd.DataFrame(metrics_list, numbers)
#display(distribution)

In [ ]:
metrics_list = []
numbers = []
desired_order = ["< 1 %", "1-49 %", ">=50 %"]

test_scores = []

for i, path in enumerate(test_parquet_paths):
    df_predictions = pd.read_parquet(path)
    dummy_clf = train_models[i]
    df_joined = pd.merge(df_predictions, csv_df, on ='slide', how = 'left')
    #print(df_joined)
    distribution = df_joined[["slide","PDL1_CATHEGORY"]]

    y_true = df_joined["PDL1_CATHEGORY"].to_numpy()
    x_list = np.ones((distribution.shape[0], 1))

    #print(distribution)

    y_pred = dummy_clf.predict(x_list)

    #val_models.append(dummy_clf)

    # Accuratezza generale
    acc = accuracy_score(y_true, y_pred)
    #print(acc)

    # Report dettagliato per classe
    report = classification_report(y_true, y_pred, digits=2, output_dict=True, zero_division=0)
    test_scores.append(report)
    print(report)

In [ ]:
import numpy as np
import pandas as pd

# Your list of dictionaries (only a few items are shown for brevity; include all in your actual code)
report = train_classification_reports

def average_report_group_with_print(group, group_index):
    print(f"\nProcessing group {group_index} (with {len(group)} items):")
    group_stats = {}
    for key in group[0]:
        if key == 'accuracy':
            values = [r[key] for r in group]
            avg_val = np.mean(values)
            print(f"  {key}: values = {values}, average = {avg_val:.4f}")
            group_stats[f"{key}_avg"] = avg_val
        else:
            # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
            for subkey in group[0][key]:
                values = [r[key][subkey] for r in group]
                avg_val = np.mean(values)
                print(f"  {key} -> {subkey}: values = {values}, average = {avg_val:.4f}")
                flat_key = f"{key}_{subkey}"
                group_stats[f"{flat_key}_avg"] = avg_val
    return group_stats

group_size = 5
averaged_stats = []

for i in range(0, len(reports), group_size):
    group = reports[i:i+group_size]
    group_index = i // group_size
    group_stats = average_report_group_with_print(group, group_index)
    averaged_stats.append(group_stats)

# Create a DataFrame from the averaged statistics.
df_train = pd.DataFrame(averaged_stats)

print("\nFinal DataFrame with averaged values:")
print(df_train)

In [ ]:
import numpy as np
import pandas as pd

# Your list of dictionaries (only a few items are shown for brevity; include all in your actual code)
report = val_classification_reports

def average_report_group_with_print(group, group_index):
    print(f"\nProcessing group {group_index} (with {len(group)} items):")
    group_stats = {}
    for key in group[0]:
        if key == 'accuracy':
            values = [r[key] for r in group]
            avg_val = np.mean(values)
            print(f"  {key}: values = {values}, average = {avg_val:.4f}")
            group_stats[f"{key}_avg"] = avg_val
        else:
            # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
            for subkey in group[0][key]:
                values = [r[key][subkey] for r in group]
                avg_val = np.mean(values)
                print(f"  {key} -> {subkey}: values = {values}, average = {avg_val:.4f}")
                flat_key = f"{key}_{subkey}"
                group_stats[f"{flat_key}_avg"] = avg_val
    return group_stats

group_size = 5
averaged_stats = []

for i in range(0, len(reports), group_size):
    group = reports[i:i+group_size]
    group_index = i // group_size
    group_stats = average_report_group_with_print(group, group_index)
    averaged_stats.append(group_stats)

# Create a DataFrame from the averaged statistics.
df_val = pd.DataFrame(averaged_stats)

print("\nFinal DataFrame with averaged values:")
print(df_val)

In [ ]:
import numpy as np
import pandas as pd

# Your list of dictionaries (only a few items are shown for brevity; include all in your actual code)
report = test_scores

def average_report_group_with_print(group, group_index):
    print(f"\nProcessing group {group_index} (with {len(group)} items):")
    group_stats = {}
    for key in group[0]:
        if key == 'accuracy':
            values = [r[key] for r in group]
            avg_val = np.mean(values)
            print(f"  {key}: values = {values}, average = {avg_val:.4f}")
            group_stats[f"{key}_avg"] = avg_val
        else:
            # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
            for subkey in group[0][key]:
                values = [r[key][subkey] for r in group]
                avg_val = np.mean(values)
                print(f"  {key} -> {subkey}: values = {values}, average = {avg_val:.4f}")
                flat_key = f"{key}_{subkey}"
                group_stats[f"{flat_key}_avg"] = avg_val
    return group_stats

group_size = 5
averaged_stats = []

for i in range(0, len(reports), group_size):
    group = reports[i:i+group_size]
    group_index = i // group_size
    group_stats = average_report_group_with_print(group, group_index)
    averaged_stats.append(group_stats)

# Create a DataFrame from the averaged statistics.
df_test = pd.DataFrame(averaged_stats)

print("\nFinal DataFrame with averaged values:")
print(df_test)

In [ ]:
df_train=df_train.reset_index()
df_val=df_val.reset_index()
df_test=df_test.reset_index()
'''
df_train.columns = ["Total number of patients", "< 1 %", "1-49 %", ">=50 %"]
df_val .columns = ["Total number of patients", "< 1 %", "1-49 %", ">=50 %"]
df_test.columns = ["Total number of patients", "< 1 %", "1-49 %", ">=50 %"]
'''


df_train_prefixed = df_train.add_suffix("_train")
df_val_prefixed   = df_val.add_suffix("_val")
df_test_prefixed  = df_test.add_suffix("_test")

df_final = pd.concat([df_train_prefixed, df_val_prefixed, df_test_prefixed], axis=1)
print(df_final)


In [ ]:
import numpy as np
import pandas as pd

# Your list of dictionaries (using your provided sample; in your code include all items)
reports = train_classification_reports

# Number of items
n = len(reports)
print(f"Total number of reports: {n}\n")

# Function to compute average and CI (CI = 2*std/sqrt(n))
def compute_avg_and_ci(values):
    avg = np.mean(values)
    std = np.std(values, ddof=1) if len(values) > 1 else 0.0
    ci = 2 * std / np.sqrt(len(values)) if len(values) > 1 else 0.0
    return avg, ci

# Dictionary to hold our flattened average and CI values
overall_stats = {}

# Process each key in the first report as a template.
for key in reports[0]:
    if key == 'accuracy':
        # Collect the accuracy values
        values = [r[key] for r in reports]
        avg, ci = compute_avg_and_ci(values)
        print(f"Metric '{key}':")
        print(f"  Values: {values}")
        print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
        overall_stats[f"{key}_avg"] = avg
        overall_stats[f"{key}_ci"] = ci
    else:
        # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
        for subkey in reports[0][key]:
            values = [r[key][subkey] for r in reports]
            avg, ci = compute_avg_and_ci(values)
            print(f"Metric '{key}' -> '{subkey}':")
            print(f"  Values: {values}")
            print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
            flat_key = f"{key}_{subkey}"
            overall_stats[f"{flat_key}_avg"] = avg
            overall_stats[f"{flat_key}_ci"] = ci

# Create a DataFrame from the overall statistics dictionary.
df = pd.DataFrame([overall_stats])
print("Final DataFrame with overall averages and CIs:")
display(df)
df.to_csv("pdl1-dummy-model-train.csv", index=True)


In [ ]:
import numpy as np
import pandas as pd

# Your list of dictionaries (using your provided sample; in your code include all items)
reports = val_classification_reports

# Number of items
n = len(reports)
print(f"Total number of reports: {n}\n")

# Function to compute average and CI (CI = 2*std/sqrt(n))
def compute_avg_and_ci(values):
    avg = np.mean(values)
    std = np.std(values, ddof=1) if len(values) > 1 else 0.0
    ci = 2 * std / np.sqrt(len(values)) if len(values) > 1 else 0.0
    return avg, ci

# Dictionary to hold our flattened average and CI values
overall_stats = {}

# Process each key in the first report as a template.
for key in reports[0]:
    if key == 'accuracy':
        # Collect the accuracy values
        values = [r[key] for r in reports]
        avg, ci = compute_avg_and_ci(values)
        print(f"Metric '{key}':")
        print(f"  Values: {values}")
        print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
        overall_stats[f"{key}_avg"] = avg
        overall_stats[f"{key}_ci"] = ci
    else:
        # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
        for subkey in reports[0][key]:
            values = [r[key][subkey] for r in reports]
            avg, ci = compute_avg_and_ci(values)
            print(f"Metric '{key}' -> '{subkey}':")
            print(f"  Values: {values}")
            print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
            flat_key = f"{key}_{subkey}"
            overall_stats[f"{flat_key}_avg"] = avg
            overall_stats[f"{flat_key}_ci"] = ci

# Create a DataFrame from the overall statistics dictionary.
df = pd.DataFrame([overall_stats])
print("Final DataFrame with overall averages and CIs:")
display(df)
df.to_csv("pdl1-dummy-model-val.csv", index=True)

In [ ]:
import numpy as np
import pandas as pd

# Your list of dictionaries (using your provided sample; in your code include all items)
reports = test_scores

# Number of items
n = len(reports)
print(f"Total number of reports: {n}\n")

# Function to compute average and CI (CI = 2*std/sqrt(n))
def compute_avg_and_ci(values):
    avg = np.mean(values)
    std = np.std(values, ddof=1) if len(values) > 1 else 0.0
    ci = 2 * std / np.sqrt(len(values)) if len(values) > 1 else 0.0
    return avg, ci

# Dictionary to hold our flattened average and CI values
overall_stats = {}

# Process each key in the first report as a template.
for key in reports[0]:
    if key == 'accuracy':
        # Collect the accuracy values
        values = [r[key] for r in reports]
        avg, ci = compute_avg_and_ci(values)
        print(f"Metric '{key}':")
        print(f"  Values: {values}")
        print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
        overall_stats[f"{key}_avg"] = avg
        overall_stats[f"{key}_ci"] = ci
    else:
        # For nested dictionary keys (like '1-49 %', '< 1 %', etc.)
        for subkey in reports[0][key]:
            values = [r[key][subkey] for r in reports]
            avg, ci = compute_avg_and_ci(values)
            print(f"Metric '{key}' -> '{subkey}':")
            print(f"  Values: {values}")
            print(f"  Average = {avg:.4f}, CI = {ci:.4f}\n")
            flat_key = f"{key}_{subkey}"
            overall_stats[f"{flat_key}_avg"] = avg
            overall_stats[f"{flat_key}_ci"] = ci

# Create a DataFrame from the overall statistics dictionary.
df = pd.DataFrame([overall_stats])
print("Final DataFrame with overall averages and CIs:")
display(df)
df.to_csv("pdl1-dummy-model-test.csv", index=True)

In [ ]:
display(df_final)

In [ ]:
df_final.to_csv("pdl1-dummy-model.csv", index=True)